In [1]:
import pandas as pd # v 1.5.3
import numpy as np  # v 1.21.5
import sklearn as sk
import statsmodels.api as sm # v 0.13.5
import matplotlib.pyplot as plt # 3.6.3
import seaborn as sns # 0. 12. 2
import warnings as ww
import statsmodels.tsa.stattools as stattools # v 0.13.5
import time


from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import export_text
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree


In [2]:
df = pd.read_csv("/Users/liamcarney9/Downloads/company_bankruptcy_data.csv")

In [3]:
# EDA

In [4]:
df.shape, df.columns, df.dtypes

((6819, 96),
 Index(['Bankrupt', ' ROA(C) before interest and depreciation before interest',
        ' ROA(A) before interest and % after tax',
        ' ROA(B) before interest and depreciation after tax',
        ' Operating Gross Margin', ' Realized Sales Gross Margin',
        ' Operating Profit Rate', ' Pre-tax net Interest Rate',
        ' After-tax net Interest Rate',
        ' Non-industry income and expenditure/revenue',
        ' Continuous interest rate (after tax)', ' Operating Expense Rate',
        ' Research and development expense rate', ' Cash flow rate',
        ' Interest-bearing debt interest rate', ' Tax rate (A)',
        ' Net Value Per Share (B)', ' Net Value Per Share (A)',
        ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons',
        ' Cash Flow Per Share', ' Revenue Per Share (Yuan ¥)',
        ' Operating Profit Per Share (Yuan ¥)',
        ' Per Share Net profit before tax (Yuan ¥)',
        ' Realized Sales Gross Profit Growth Rate

In [5]:
df.isna().sum()

Bankrupt                                                    0
 ROA(C) before interest and depreciation before interest    0
 ROA(A) before interest and % after tax                     0
 ROA(B) before interest and depreciation after tax          0
 Operating Gross Margin                                     0
                                                           ..
 Liability to Equity                                        0
 Degree of Financial Leverage (DFL)                         0
 Interest Coverage Ratio (Interest expense to EBIT)         0
 Net Income Flag                                            0
 Equity to Liability                                        0
Length: 96, dtype: int64

In [6]:
# since the above code only shows missing values for a certain amount of columns, this code below will read 'True' if
# every column has zero missing value. As seen below, it reads 'True'

missing = (df.isnull().sum() == 0).all()
print(missing)

True


In [7]:
df.describe()

,Bankrupt,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,...,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.0,6819.000000
mean,0.032263,0.505180,0.558625,0.553589,0.607948,0.607929,0.998755,0.797190,0.809084,0.303623,...,0.807760,1.862942e+07,0.623915,0.607946,0.840402,0.280365,0.027541,0.565358,1.0,0.047578
std,0.176710,0.060686,0.065620,0.061595,0.016934,0.016916,0.013010,0.012869,0.013601,0.011163,...,0.040332,3.764501e+08,0.012290,0.016934,0.014523,0.014463,0.015668,0.013214,0.0,0.050014
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
25%,0.000000,0.476527,0.535543,0.527277,0.600445,0.600434,0.998969,0.797386,0.809312,0.303466,...,0.796750,9.036205e-04,0.623636,0.600443,0.840115,0.276944,0.026791,0.565158,1.0,0.024477
50%,0.000000,0.502706,0.559802,0.552278,0.605997,0.605976,0.999022,0.797464,0.809375,0.303525,...,0.810619,2.085213e-03,0.623879,0.605998,0.841179,0.278778,0.026808,0.565252,1.0,0.033798
75%,0.000000,0.535563,0.589157,0.584105,0.613914,0.613842,0.999095,0.797579,0.809469,0.303585,...,0.826455,5.269777e-03,0.624168,0.613913,0.842357,0.281449,0.026913,0.565725,1.0,0.052838
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,9.820000e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000


In [8]:
df.head()

,Bankrupt,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [9]:
correlation = df.corr()
print(correlation)

                                                    Bankrupt  \
Bankrupt                                            1.000000   
 ROA(C) before interest and depreciation before... -0.260807   
 ROA(A) before interest and % after tax            -0.282941   
 ROA(B) before interest and depreciation after tax -0.273051   
 Operating Gross Margin                            -0.100043   
...                                                      ...   
 Liability to Equity                                0.166812   
 Degree of Financial Leverage (DFL)                 0.010508   
 Interest Coverage Ratio (Interest expense to E... -0.005509   
 Net Income Flag                                         NaN   
 Equity to Liability                               -0.083048   

                                                     ROA(C) before interest and depreciation before interest  \
Bankrupt                                                                                    -0.260807          
 ROA(C)

In [10]:
class_distribution = df['Bankrupt'].value_counts()
print(class_distribution)

0    6599
1     220
Name: Bankrupt, dtype: int64


In [11]:
# the code above shows that the data is imbalanced, with 0 being more prevelant than 1, so we will oversample

majority = df[df['Bankrupt']==0]
minority = df[df['Bankrupt']==1]

majority_undersampled = resample(majority, replace = True, n_samples=len(minority))
undersampled = pd.concat([minority, majority_undersampled])

undersampled.describe()

,Bankrupt,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,...,440.000000,4.400000e+02,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.0,440.000000
mean,0.500000,0.464841,0.511856,0.510978,0.603141,0.603154,0.998880,0.797035,0.808905,0.303092,...,0.775421,4.550000e+07,0.623772,0.603139,0.833633,0.286856,0.027879,0.565358,1.0,0.034435
std,0.500569,0.083734,0.102087,0.090867,0.014015,0.013981,0.000538,0.002318,0.003006,0.003669,...,0.071781,5.976066e+08,0.004772,0.014015,0.054076,0.048955,0.012394,0.007052,0.0,0.047282
min,0.000000,0.024277,0.000000,0.033514,0.532906,0.532906,0.991888,0.757216,0.761550,0.235090,...,0.411809,1.106360e-04,0.587512,0.532907,0.000000,0.000000,0.020760,0.506547,1.0,0.003946
25%,0.000000,0.432592,0.475292,0.480178,0.596616,0.596645,0.998868,0.797107,0.809050,0.303293,...,0.758873,9.556008e-04,0.623347,0.596614,0.836160,0.278023,0.026629,0.564475,1.0,0.018046
50%,0.500000,0.474650,0.535352,0.525430,0.602045,0.602055,0.998972,0.797381,0.809307,0.303456,...,0.795995,2.763487e-03,0.623769,0.602045,0.840076,0.281622,0.026791,0.565160,1.0,0.023789
75%,1.000000,0.506142,0.564817,0.556895,0.609358,0.609532,0.999045,0.797477,0.809387,0.303538,...,0.812662,7.312926e-03,0.624076,0.609355,0.841527,0.286755,0.026878,0.565739,1.0,0.037867
max,1.000000,0.780139,0.875600,0.869158,0.665151,0.665151,0.999592,0.803263,0.814604,0.313087,...,0.934402,9.170000e+09,0.679654,0.665149,1.000000,1.000000,0.264577,0.624147,1.0,0.798122


In [12]:
undersampled.Bankrupt.value_counts()

1    220
0    220
Name: Bankrupt, dtype: int64

In [13]:
# We can see above that the different values of 'Bankrupt' are not equal in proportion after oversampling. I did not
# want to undersample because there were only 220 observations in the underrepresented group, and wanted more than 440
# total observations

In [14]:
# PART 1 - #1

In [15]:
X = df.drop('Bankrupt', axis=1)
y = df['Bankrupt']

# Split the dataset into training and testing sets
X_traindf, X_testdf, y_traindf, y_testdf = train_test_split(X, y, test_size=0.2, random_state=42)

rf_modeldf = RandomForestClassifier(random_state=42)

rf_modeldf.fit(X_traindf, y_traindf)

# Make predictions on the test data
y_preddf = rf_modeldf.predict(X_testdf)

accuracydf = accuracy_score(y_testdf, y_preddf)
precisiondf = precision_score(y_testdf, y_preddf)
recalldf = recall_score(y_testdf, y_preddf)

print(f"Accuracy: {accuracydf:.4f}")
print(f"Precision: {precisiondf:.4f}")
print(f"Recall: {recalldf:.4f}")

Accuracy: 0.9677
Precision: 0.8182
Recall: 0.1765


In [16]:
# looking at the accuracy, it appears that the random forest model did a very good job overall, supported by its 
# rate in accuracy of 96.77%. It also had a good precision rate of 81.82%, however the recall was low at 17.65%.

In [17]:
# PART 1 #2

In [62]:
rf_model_new = RandomForestClassifier(
    n_estimators=100, # CHANGING PARAMETERS
    max_depth=10,      
    min_samples_split=2, 
    min_samples_leaf=1,   
    criterion='entropy',    # Change crtierion from gini to entropy
    random_state=42
)

rf_model_new.fit(X_traindf, y_traindf)

# Make predictions on the test data
y_pred_new = rf_model_new.predict(X_testdf)

accuracy_new = accuracy_score(y_testdf, y_pred_new)
precision_new = precision_score(y_testdf, y_pred_new)
recall_new = recall_score(y_testdf, y_pred_new)

print(f"Accuracy new: {accuracy_new:.4f}")
print(f"Precision new: {precision_new:.4f}")
print(f"Recall new: {recall_new:.4f}")

Accuracy new: 0.9663
Precision new: 0.7273
Recall new: 0.1569


In [19]:
# This model gives us error metrics that appear to be slightly worse than the original random forest model. All error 
# metrics are worse than that of the default parameters, especially the precision. That said, the RF model with 
# default parameters gave us better metrics, and that is the model we should use. 

In [20]:
# PART 1 #3

In [21]:
# default RF model
start_time = time.time()

rf_modeldf = RandomForestClassifier(random_state=42)
rf_modeldf.fit(X_traindf, y_traindf)

end_time = time.time()
elapsed_time = end_time - start_time
y_pred_default = rf_modeldf.predict(X_testdf)

# new custom model
start_time_new = time.time()

rf_model_new = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1, random_state=42)

rf_model_new.fit(X_traindf, y_traindf)

end_time_new = time.time()
elapsed_time_new = end_time_new - start_time_new
y_pred_new = rf_model_new.predict(X_testdf)


print(f"Elapsed Time (Default Parameters): {elapsed_time:.4f} seconds")
print(f"Elapsed Time (New Parameters): {elapsed_time_new:.4f} seconds")


Elapsed Time (Default Parameters): 2.4217 seconds
Elapsed Time (New Parameters): 2.2716 seconds


In [22]:
# The elapsed time of the model with custom parameters runs quicker than that of default parameters by approx 1 second

In [23]:
# PART 1 #4

In [24]:
rf_model_new2 = RandomForestClassifier(random_state=42, criterion='entropy')

rf_model_new2.fit(X_traindf, y_traindf)

# Make predictions on the test data
y_pred_new2 = rf_model_new2.predict(X_testdf)

accuracy_new2 = accuracy_score(y_testdf, y_pred_new2)
precision_new2 = precision_score(y_testdf, y_pred_new2)
recall_new2 = recall_score(y_testdf, y_pred_new2)

print(f"Accuracy new: {accuracy_new2:.4f}")
print(f"Precision new: {precision_new2:.4f}")
print(f"Recall new: {recall_new2:.4f}")

Accuracy new: 0.9663
Precision new: 0.7273
Recall new: 0.1569


In [25]:
start_time_new2 = time.time()

rf_model_new2 = RandomForestClassifier(random_state=42, criterion='entropy')

rf_model_new2.fit(X_traindf, y_traindf)

end_time_new2 = time.time()
elapsed_time_new2 = end_time_new2 - start_time_new2
y_pred_new2 = rf_model_new2.predict(X_testdf)


print(f"Elapsed Time (New Parameters): {elapsed_time_new2:.4f} seconds")

Elapsed Time (New Parameters): 1.8823 seconds


In [26]:
# results in question 2 show us that error metrics are worse, however that was when several parameters were changed. 
# This model, however, only changes the criterion from gini to entropy. Looking at the error metrics now, we can see 
# that gini and entropy have similar metrics, however all of entropy's metrics are worse. In particular, the precision
# is significantly worse. Although it does have a quicker elapsed time, I believe it is wiser to use gini as criterion

In [27]:
# PART 1 #5

In [60]:
# USING RF MODEL

first_tree_index = 0
first_tree = rf_modeldf.estimators_[first_tree_index]

# Display the decision tree
tree_rules = export_text(first_tree, feature_names=list(X.columns))
print(tree_rules)

|---  Net Income to Stockholder's Equity <= 0.83
|   |---  Net Worth Turnover Rate (times) <= 0.03
|   |   |---  Borrowing dependency <= 0.40
|   |   |   |---  Interest Coverage Ratio (Interest expense to EBIT) <= 0.56
|   |   |   |   |---  Realized Sales Gross Margin <= 0.59
|   |   |   |   |   |--- class: 0.0
|   |   |   |   |---  Realized Sales Gross Margin >  0.59
|   |   |   |   |   |--- class: 1.0
|   |   |   |---  Interest Coverage Ratio (Interest expense to EBIT) >  0.56
|   |   |   |   |---  Accounts Receivable Turnover <= 0.00
|   |   |   |   |   |---  Accounts Receivable Turnover <= 0.00
|   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |---  Accounts Receivable Turnover >  0.00
|   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |---  Accounts Receivable Turnover >  0.00
|   |   |   |   |   |---  Inventory/Working Capital <= 0.28
|   |   |   |   |   |   |---  Realized Sales Gross Profit Growth Rate <= 0.02
|   |   |   |   |   |   |   |--- class: 1.0
|   |   | 

In [61]:
# This decision rules tree does make sense. Decision trees can provide insights into the importance of different 
# features in making predictions. Features that are higher up in the tree and closer to the root node are generally 
# more important in making decisions, we will look into this in the next question. But generally, each node is split
# at a certain threshold, and a decision is made as to what class an observation belongs to, continuing through to
# each variable. 

In [28]:
# PART 1 #6

In [29]:
feature_importance = rf_modeldf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by = 'Importance', ascending = False)
print('Feature Importance:')
print(feature_importance_df)

Feature Importance:
                                     Feature  Importance
89        Net Income to Stockholder's Equity    0.030829
29                     Net Value Growth Rate    0.026052
39                      Borrowing dependency    0.022574
64                    Working Capital/Equity    0.019791
18   Persistent EPS in the Last Four Seasons    0.019468
..                                       ...         ...
40          Contingent liabilities/Net worth    0.005294
44                      Total Asset Turnover    0.003905
14                              Tax rate (A)    0.002235
84                     Liability-Assets Flag    0.000250
93                           Net Income Flag    0.000000

[95 rows x 2 columns]


In [30]:
# using the random forest model with default parameters, feature importance tells us that Net Income to Stockholder's
# Equity was the most important feature to predicting either Bankrupt or not. Then NVGR then Borrowing dependency, etc

In [31]:
# PART 1 #7

In [32]:
# on average, I think it varies from person to person, but as long as a person is willing to wait slightly longer, I
# think it is worth it to fit the model with gini and default parameters. You may lose a second or two in time 
# efficiency, but you can have slightly greater accuracy and recall, and greatly improve precision.

In [33]:
# SVM #1

In [35]:
X = undersampled.drop('Bankrupt', axis=1)
y = undersampled['Bankrupt']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# Default parameters svm model
svm_model = SVC(random_state=42)

# Fit the model to the same training data used previously
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)

print(f"Accuracy SVM: {accuracy_svm:.4f}")
print(f"Precision SVM: {precision_svm:.4f}")
print(f"Recall SVM: {recall_svm:.4f}")

Accuracy SVM: 0.6250
Precision SVM: 0.7250
Recall SVM: 0.5686


In [37]:
# It appears that SVM's accuracy is significantly lower than RF models, has similar precision, and makes up for its
# lack of accuracy with greatly improved recall. 

In [38]:
# SVM #2

In [ ]:
# change SVM kernel to "linear"
svm_model_linear = SVC(kernel='linear', random_state=42)

svm_model_linear.fit(X_train, y_train)

y_pred_svm_linear = svm_model_linear.predict(X_test)

# Evaluate the SVM model with the linear kernel
accuracy_svm_linear = accuracy_score(y_test, y_pred_svm_linear)
precision_svm_linear = precision_score(y_test, y_pred_svm_linear)
recall_svm_linear = recall_score(y_test, y_pred_svm_linear)

print(f"Accuracy SVM Linear: {accuracy_svm_linear:.4f}")
print(f"Precision SVM Linear: {precision_svm_linear:.4f}")
print(f"Recall SVM Linear: {recall_svm_linear:.4f}")

In [39]:
# upon changing the the kernel from rbf to linear, the linear kernel SVM model does not run. If it does, it takes an
# indefinite aamount of time to finish running. 

In [40]:
# SVM #3

In [41]:
# looking at #1, we see that the SVM model with rbf kernel has pretty good metrics (with much higher recall than RF). 
# On the other hand, SVM with a linear kernel does not show any metrics as the cell does not fully run.

In [42]:
# SVM #4

In [ ]:
# SVM
start_time_svm = time.time()

svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)

end_time_svm = time.time()
elapsed_time_svm = end_time_svm - start_time_svm
y_pred_svm = svm_model.predict(X_test)

print(f"Elapsed Time (SVM): {elapsed_time_svm:.4f} seconds")

# SVM Linear
start_time_svml = time.time()

svm_model_linear = SVC(kernel='linear', random_state=42)

svm_model_linear.fit(X_train, y_train)

end_time_svml = time.time()
elapsed_time_svml = end_time_svml - start_time_svml
y_pred_svml = svm_model_linear.predict(X_test)


print(f"Elapsed Time (SVM Linear): {elapsed_time_svml:.4f} seconds")

In [43]:
# Because the SVM Linear model will not run, we can not see the elapsed times exactly, however, we know that the SVM
# model runs while SVM Linear does not, so SVM is much quicker. 

In [ ]:
# PART 2 - #1

In [50]:
# RANDOM FOREST UNDERSAMPLED

X = undersampled.drop('Bankrupt', axis=1)
y = undersampled['Bankrupt']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

start_time_rfu = time.time()

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
end_time_rfu = time.time()
elapsed_time_rfu = end_time_rfu - start_time_rfu

# Make predictions on the test data
y_pred = rf_model.predict(X_test)

print(f"Elapsed Time (RF Undersampled): {elapsed_time_rfu:.4f} seconds")

Elapsed Time (RF Undersampled): 0.1979 seconds


In [52]:
# SVM Undersampled
start_time_svmu = time.time()

# Default parameters svm model UNDERSAMPLED
svmu_model = SVC(random_state=42)

# Fit the model to the same training data used previously
svmu_model.fit(X_train, y_train)

end_time_svmu = time.time()
elapsed_time_svmu = end_time_svmu - start_time_svmu

y_pred_svmu = svmu_model.predict(X_test)

print(f"Elapsed Time (SVM Undersampled): {elapsed_time_svmu:.4f} seconds")

Elapsed Time (SVM Undersampled): 0.0136 seconds


In [53]:
# The SVM undersampled model produces a model prediction much quicker than the RF undersampled. RF Undersampled took
# 0.1979 seconds and 0.0136 seeconds for SVM undersampled

In [46]:
# PART 2 - #2

In [55]:
# Random Forest
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Random Forest Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# SVM 
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)

print(" ")
print("SVM Metrics:")
print(f"Accuracy SVM: {accuracy_svm:.4f}")
print(f"Precision SVM: {precision_svm:.4f}")
print(f"Recall SVM: {recall_svm:.4f}")

Random Forest Metrics:
Accuracy: 0.8409
Precision: 0.8776
Recall: 0.8431
 
SVM Metrics:
Accuracy SVM: 0.6250
Precision SVM: 0.7250
Recall SVM: 0.5686


In [56]:
# Seen from the metrics above, the Random Forest metrics are much higher than that of the SVM metrics. However, we 
# come across the question of time vs error. We can obtain much better error metrics by taking slightly longer, or 
# vice versa if we are in a need for quicker time.  